In [ ]:
import h5py
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from scipy import sparse
import lightgbm as lgb
import gc
import os
import joblib


In [ ]:
# Параметры
IMG_H5 = "image_features.h5"
TRAIN_CSV = "ml_ozon_сounterfeit_train.csv"
TEXT_COLS = ["name_rus", "description"]
CAT_COLS = ["brand_name", "CommercialTypeName4"]
TARGET = "resolution"
ID_COL = "id"
N_FOLDS = 5
RANDOM_STATE = 42

In [ ]:
print("Загрузка данных...")

# 1. ОПТИМИЗАЦИЯ: Загружаем только нужные данные из CSV
df = pd.read_csv(TRAIN_CSV)
df[TARGET] = df[TARGET].astype(int)

print(f"Размер датасета: {df.shape}")
print(f"Использование памяти до оптимизации: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Оптимизация типов данных для экономии памяти
def optimize_dtypes(df):
    """Оптимизируем типы данных для экономии памяти"""
    for col in df.columns:
        if df[col].dtype == 'object':
            continue
        elif df[col].dtype == 'int64':
            if df[col].min() >= 0:
                if df[col].max() < 255:
                    df[col] = df[col].astype('uint8')
                elif df[col].max() < 65535:
                    df[col] = df[col].astype('uint16')
                elif df[col].max() < 4294967295:
                    df[col] = df[col].astype('uint32')
            else:
                if df[col].min() > -128 and df[col].max() < 127:
                    df[col] = df[col].astype('int8')
                elif df[col].min() > -32768 and df[col].max() < 32767:
                    df[col] = df[col].astype('int16')
                elif df[col].min() > -2147483648 and df[col].max() < 2147483647:
                    df[col] = df[col].astype('int32')
        elif df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
    return df

df = optimize_dtypes(df)
print(f"Использование памяти после оптимизации: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


In [ ]:
# 2. ОПТИМИЗАЦИЯ: Предварительная подготовка колонок
exclude = {ID_COL, "ItemID", "SellerID", TARGET}
num_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c not in exclude]
cat_cols = [c for c in CAT_COLS if c in df.columns]

print("Numeric cols:", num_cols)
print("Cat cols:", cat_cols)
print("Text cols:", [c for c in TEXT_COLS if c in df.columns])

# 3. ОПТИМИЗАЦИЯ: Подготавливаем категориальные признаки
for c in cat_cols:
    counts = df[c].fillna("##NA##").value_counts(dropna=False).to_dict()
    df[c + "_freq"] = df[c].fillna("##NA##").map(counts).astype('float32')

cat_freq_cols = [c + "_freq" for c in cat_cols]

# 4. ОПТИМИЗАЦИЯ: Подготавливаем числовые признаки
X_num = df[num_cols].copy().astype('float32')
imp = SimpleImputer(strategy="median")
X_num_imp = imp.fit_transform(X_num).astype('float32')
del X_num
gc.collect()

scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X_num_imp).astype('float32')
del X_num_imp
gc.collect()

# 5. ОПТИМИЗАЦИЯ: Подготавливаем текстовые признаки
print("Подготовка текстовых признаков...")
texts = []
for i, row in df.iterrows():
    pieces = []
    for c in TEXT_COLS:
        if c in df.columns:
            val = row.get(c, "")
            pieces.append(str(val) if pd.notna(val) else "")
    texts.append(" ".join(pieces))

# Уменьшаем количество признаков для TF-IDF
tf = TfidfVectorizer(max_features=80_000, ngram_range=(1,2), dtype=np.float32)
X_text = tf.fit_transform(texts)
del texts
gc.collect()

print(f"Размер TF-IDF матрицы: {X_text.shape}")

# 6. ОПТИМИЗАЦИЯ: Подготавливаем категориальные частоты
if cat_freq_cols:
    X_catfreq = df[cat_freq_cols].fillna(0).values.astype('float32')
else:
    X_catfreq = np.zeros((len(df), 0), dtype='float32')

In [ ]:
# 7. ОПТИМИЗАЦИЯ: Пакетная загрузка изображений
print("Загрузка изображений...")

def load_image_features_batch(h5_path, item_ids, batch_size=10000):
    """Загружаем изображения батчами для экономии памяти"""
    with h5py.File(h5_path, "r") as f:
        img_features = f["features"]
        img_ids = f["ids"][:]
        
        # Создаем маппинг
        id2idx = {int(i): j for j, i in enumerate(img_ids)}
        
        # Определяем размерность фичей
        feature_dim = img_features.shape[1]
        n_items = len(item_ids)
        
        # Инициализируем результирующую матрицу
        result = np.zeros((n_items, feature_dim), dtype='float32')
        
        # Обрабатываем батчами
        for start_idx in range(0, n_items, batch_size):
            end_idx = min(start_idx + batch_size, n_items)
            batch_item_ids = item_ids[start_idx:end_idx]
            
            for local_idx, item_id in enumerate(batch_item_ids):
                global_idx = start_idx + local_idx
                if item_id in id2idx:
                    result[global_idx] = img_features[id2idx[item_id]]
                # Иначе остается нулевой вектор (уже инициализирован)
            
            if (start_idx // batch_size) % 10 == 0:
                print(f"Обработано {end_idx}/{n_items} изображений")
    
    return result

# Загружаем изображения батчами
img_vectors = load_image_features_batch(IMG_H5, df["ItemID"].values)
print(f"img_vectors shape: {img_vectors.shape}")

In [ ]:
# 8. Создаем разреженные матрицы для экономии памяти
X_img_sparse = sparse.csr_matrix(img_vectors, dtype='float32')
X_num_sparse = sparse.csr_matrix(X_num_scaled, dtype='float32')
X_cat_sparse = sparse.csr_matrix(X_catfreq, dtype='float32')

# Освобождаем память
del img_vectors, X_num_scaled, X_catfreq
gc.collect()

In [ ]:
# 9. Объединяем все признаки
print("Объединение признаков...")
X = sparse.hstack([X_text, X_cat_sparse, X_num_sparse, X_img_sparse], format="csr")
print(f"Final feature matrix shape: {X.shape}")

# Освобождаем память
del X_text, X_cat_sparse, X_num_sparse, X_img_sparse
gc.collect()

In [ ]:
# 10. Обучение с управлением памяти
y = df[TARGET].values.astype('uint8')
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)

oof = np.zeros(len(df), dtype='uint8')
oof_probs = np.zeros(len(df), dtype='float32')
models = []
best_thresholds = []
fold_scores = []

print("Начинаем обучение...")

for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), 1):
    print(f'Обработка fold {fold}...')
    
    X_tr, X_va = X[tr_idx], X[va_idx]
    y_tr, y_va = y[tr_idx], y[va_idx]
    
    # Параметры LightGBM для экономии памяти
    clf = lgb.LGBMClassifier(
        n_estimators=1000,  # Уменьшено для экономии памяти
        learning_rate=0.1,   # Увеличено чтобы компенсировать меньше деревьев
        num_leaves=31,
        n_jobs=-1,
        random_state=RANDOM_STATE,
        class_weight='balanced',
        max_bin=255,        # Уменьшено для экономии памяти
        feature_fraction=0.8,  # Сэмплируем признаки
        bagging_fraction=0.8,  # Сэмплируем объекты
        verbosity=-1
    )
    
    # Обучение с early stopping
    clf.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric='binary_logloss',
        callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)]
    )
    
    models.append(clf)
    
    # Предсказания
    pval = clf.predict_proba(X_va)[:, 1].astype('float32')
    oof_probs[va_idx] = pval
    
    # Поиск оптимального порога
    ths = np.linspace(0.01, 0.99, 99)
    best_th, best_f1 = 0.5, 0.0
    for t in ths:
        f = f1_score(y_va, (pval >= t).astype('uint8'))
        if f > best_f1:
            best_f1 = f
            best_th = t
    
    best_thresholds.append(best_th)
    oof[va_idx] = (pval >= best_th).astype('uint8')
    f1_fold = f1_score(y_va, oof[va_idx])
    fold_scores.append(f1_fold)
    
    print(f"Fold {fold} -> best_th={best_th:.3f} | val F1={f1_fold:.4f}")
    
    # Принудительная очистка памяти
    gc.collect()

In [ ]:
print("\n=== РЕЗУЛЬТАТЫ ===")
print("CV folds F1:", fold_scores)
print("Mean CV F1:", np.mean(fold_scores))
print("OOF F1:", f1_score(y, oof))

# Сохранение моделей
print("\nСохранение моделей...")
os.makedirs("models", exist_ok=True)
for i, m in enumerate(models):
    joblib.dump(m, f"models/lgb_fold{i+1}.pkl")

joblib.dump(tf, "models/tfidf.pkl")
joblib.dump(imp, "models/imputer.pkl")
joblib.dump(scaler, "models/scaler.pkl")

# Сохраняем мэппинг частот категорий
cat_counts = {c: df[c].fillna("##NA##").value_counts(dropna=False).to_dict() 
              for c in cat_cols}
joblib.dump(cat_counts, "models/cat_counts.pkl")
joblib.dump(best_thresholds, "models/best_thresholds.pkl")

print("Готово!")

# Построим важность признаков для последней модели
try:
    lgb.plot_importance(models[-1], max_num_features=30)
except:
    print("Не удалось построить график важности признаков")